In [1]:
dataset <- "ARPAFVG"
state <- "Friuli-Venezia Giulia"
match_dist <- 15000


In [2]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)

source("src/database/tools.R")
source("src/database/insert.R")
source("src/database/write.R")
source("src/database/startup.R")
source("src/database/plot.R")
source("src/database/query/spatial.R")
source("src/database/query/data.R")
source("src/merging/analysis.R")
source("src/merging/display.R")
source("src/merging/combining.R")
source("src/merging/tagging.R")

conns <- load_dbs()
state_boundaries <- query_boundary(conns$stations, state, "state")


Linking to GEOS 3.11.0, GDAL 3.5.3, PROJ 9.1.0; sf_use_s2() is TRUE



In [3]:
ds_meta <- query_checkpoint_meta(dataset, "qc1", conns$data) |>
    collect()
nat_meta <- query_checkpoint_meta(c("SCIA", "ISAC"), "qc1", conns$data) |>
    collect() |>
    st_md_to_sf() |>
    st_filter(state_boundaries |> st_buffer(dist = 1000), .predicate = st_within) |>
    st_drop_geometry() |>
    filter(!state %in% c("Veneto"), !(network %in% c("Sinottica", "Mareografica")))
metadata <- bind_rows(ds_meta, nat_meta) |>
    arrange(dataset, sensor_key) |>
    mutate(key = row_number())
metadata_db <- copy_to(conns$data, metadata, "metas_tmp", overwrite = TRUE)

sensor_matches <- close_matches(metadata_db, match_dist, conns$stations)

data <- query_checkpoint_data(c(dataset, "SCIA", "ISAC"), "qc1", conns$data) |>
    filter(valid) |>
    inner_join(metadata_db |> select(dataset, sensor_key, key), by = c("dataset", "sensor_key")) |>
    select(!c(starts_with("qc_"), valid))

var_matches <- series_matches(data, sensor_matches, metadata_db)


In [4]:
analysis <- series_matches_analysis(var_matches, data, metadata_db, matches_offsets = c(0L))
analysis |> write_xlsx_analysis(file.path("notebooks", "integrazioni_regionali", "FVG", "raw_analysis.xlsx"))


In [18]:
source("notebooks/integrazioni_regionali/FVG/procedures/match_taggers.R")
analysis |>
    tag_same_series() |>
    default_logic(FALSE) |> 
    write_xlsx_analysis(file.path("notebooks", "integrazioni_regionali", "FVG", "tagged_analysis.xlsx"), starts_with("tag_"))


In [12]:
library(tidyr, warn.conflicts = FALSE)
data |> arrange(dataset, sensor_key, variable, date) |> collect() |> nest(.by = c(dataset, sensor_key, variable)) |> print()

# A tibble: 861 x 4
   dataset sensor_key variable data                
   <chr>        <int>    <int> <list>              
 1 ARPAFVG          1       -1 <tibble [5,487 x 3]>
 2 ARPAFVG          1        1 <tibble [5,482 x 3]>
 3 ARPAFVG          2       -1 <tibble [5,487 x 3]>
 4 ARPAFVG          2        1 <tibble [5,483 x 3]>
 5 ARPAFVG          4       -1 <tibble [5,436 x 3]>
 6 ARPAFVG          4        1 <tibble [5,435 x 3]>
 7 ARPAFVG          5       -1 <tibble [3,643 x 3]>
 8 ARPAFVG          5        1 <tibble [3,643 x 3]>
 9 ARPAFVG          6       -1 <tibble [5,900 x 3]>
10 ARPAFVG          6        1 <tibble [5,890 x 3]>
# i 851 more rows
